# Preselection of CC vertices in fiducial volume

In [ ]:
import ROOT

In [ ]:
ROOT.gROOT.ProcessLine('#include "ShipMCTrack.h"')
ROOT.gROOT.ProcessLine('#include "AdvTargetPoint.h"')

In [ ]:
df = ROOT.ROOT.RDataFrame("cbmsim", "numu_dig.root")

In [ ]:
columns_orig = df.GetColumnNames()

## Helper functions

In [ ]:
# JIT a C++ function from Python
ROOT.gInterpreter.Declare("""
template<typename T>
ROOT::RVec<T*> vectorise(const TClonesArray& xs) {
    ROOT::RVec<T*> v{};
    for (auto x: xs) {
        v.emplace_back(static_cast<T*>(x));
    }
    return v;
}
""")

In [ ]:
# JIT a C++ function from Python
ROOT.gInterpreter.Declare("""
auto GetStart(ShipMCTrack* t) {
    if (t) {
        return make_tuple(t->GetStartX(), t->GetStartY(), t->GetStartZ());
    } else {
        return make_tuple(0.,0.,0.);
    }
}
""")

In [ ]:
# JIT a C++ function from Python
ROOT.gInterpreter.Declare("""
ROOT::RVec<int> motherIDs(ROOT::RVec<ShipMCTrack*> xs) {
    ROOT::RVec<int> v{};
    for (auto x: xs) {
        v.push_back(x->GetMotherId());
    }
    return v;
}
""")

In [ ]:
# JIT a C++ function from Python
ROOT.gInterpreter.Declare("""
ROOT::RVec<bool> isMuon(ROOT::RVec<ShipMCTrack*> xs) {
    ROOT::RVec<int> v{};
    for (auto x: xs) {
        v.push_back(std::abs(x->GetPdgCode()) == 13);
    }
    return v;
}
""")

In [ ]:
df = df.Define("MCTracks", "vectorise<ShipMCTrack>(MCTrack)")

In [ ]:
df = df.Define("IsMuon", "isMuon(MCTracks)")

In [ ]:
df = df.Define("PrimaryMCTrack", "MCTracks[0]")

In [ ]:
df = df.Define("MotherIDs", "motherIDs(MCTracks)")

In [ ]:
h1 = df.Histo1D("MotherIDs")

In [ ]:
c1 = ROOT.TCanvas()
h1.Draw()
c1.Draw()

In [ ]:
df = df.Define("SecondaryMCTracks", "MCTracks[MotherIDs == 0]")

In [ ]:
h2 = df.Define("SecondaryMCTrackMothers", "motherIDs(SecondaryMCTracks)").Histo1D(
    "SecondaryMCTrackMothers"
)

In [ ]:
c2 = ROOT.TCanvas()
h2.Draw()
c2.Draw()

## Identify secondary muon

In [ ]:
df = df.Define("SecondaryMuons", "MCTracks[MotherIDs == 0 && IsMuon]")

In [ ]:
h3 = df.Define("nSecondaryMuons", "std::size(SecondaryMuons)").Histo1D(
    "nSecondaryMuons"
)

In [ ]:
c3 = ROOT.TCanvas()
h3.Draw()
c3.Draw()

In [ ]:
# df = df.Filter("std::size(SecondaryMuons) == 1", "Secondary muon")
df = df.Define("CC_cut", "std::size(SecondaryMuons) == 1")

In [ ]:
df = df.Define("PrimaryPdg", "PrimaryMCTrack->GetPdgCode()")

In [ ]:
h = df.Histo1D("PrimaryPdg")

In [ ]:
c = ROOT.TCanvas()
h.Draw()
c.Draw()

## Find vertex position (secondary muon start)

In [ ]:
df = df.Define("Vertex", "GetStart(SecondaryMuons[0])")

In [ ]:
h4 = (
    df.Define("vertex_x", "std::get<0>(Vertex)")
    .Define("vertex_y", "std::get<1>(Vertex)")
    .Define("vertex_z", "std::get<2>(Vertex)")
    .Histo2D(
        ("vertex", "vertex location; x [cm]; y[cm]", 100, -70, 10, 100, 0, 80),
        "vertex_x",
        "vertex_y",
    )
)

In [ ]:
c4 = ROOT.TCanvas()
h4.Draw("colz")
c4.Draw()

In [ ]:
h5 = df.Define("vertex_z", "std::get<2>(Vertex)").Histo1D("vertex_z")

In [ ]:
c5 = ROOT.TCanvas()
h5.Draw()
c5.Draw()

In [ ]:
# Check whether vertex is in the fiducial volume

In [ ]:
h6 = (
    df.Define("AdvTargetPointX", "AdvTargetPoint.fX")
    .Define("AdvTargetPointY", "AdvTargetPoint.fY")
    .Histo2D(
        ("points", "point location; x [cm]; y[cm]", 100, -50, 0, 100, 10, 60),
        "AdvTargetPointX",
        "AdvTargetPointY",
    )
)

In [ ]:
c6 = ROOT.TCanvas()
h6.Draw("colz")
c6.Draw()

In [ ]:
df = df.Define("detids", "AdvTargetPoint.fDetectorID")

In [ ]:
df = df.Define(
    "isvertical",
    "return Map(detids, [](int detid){return int(detid >> 14) % 2;});",
    ["detids"],
)

In [ ]:
df = df.Define("isnotvertical", "!isvertical")

In [ ]:
h7 = df.Define("AdvTargetPointY", "AdvTargetPoint.fY").Histo1D(
    "AdvTargetPointY", "isvertical"
)

In [ ]:
c7 = ROOT.TCanvas()
h7.Draw()
c7.Draw()

In [ ]:
h8 = df.Define("AdvTargetPointY", "AdvTargetPoint.fY").Histo1D(
    "AdvTargetPointY", "isnotvertical"
)

In [ ]:
c8 = ROOT.TCanvas()
h8.Draw()
c8.Draw()

In [ ]:
h9 = (
    df.Define("AdvTargetPointX", "AdvTargetPoint.fX")
    .Define("AdvTargetPointY", "AdvTargetPoint.fY")
    .Histo2D(
        ("points", "point location; x [cm]; y[cm]", 1000, -50, 0, 1000, 10, 60),
        "AdvTargetPointX",
        "AdvTargetPointY",
        "isnotvertical",
    )
)

In [ ]:
c9 = ROOT.TCanvas()
h9.Draw("colz")
c9.Draw()

In [ ]:
h10 = (
    df.Define("AdvTargetPointX", "AdvTargetPoint.fX")
    .Define("AdvTargetPointY", "AdvTargetPoint.fY")
    .Histo2D(
        ("points", "point location; x [cm]; y[cm]", 1000, -50, 0, 1000, 10, 60),
        "AdvTargetPointX",
        "AdvTargetPointY",
        "isvertical",
    )
)

In [ ]:
c10 = ROOT.TCanvas()
h10.Draw("colz")
c10.Draw()

In [ ]:
# JIT a C++ function from Python
ROOT.gInterpreter.Declare("""
bool is_fiducial_x(const std::tuple<double,double,double> &vtx) {
    auto x = std::get<0>(vtx);
    auto vertical = (x < -3 && x > -11) || (x < -15 && x > -23) || (x < -27 && x > -35) || (x < -39 && x > -47);
    auto horizontal = (x < -6 && x > -14) || (x < -15 && x > -23) || (x < -27 && x > -35) || (x < -37 && x > -44);
    return horizontal && vertical;
}
""")

In [ ]:
ROOT.gInterpreter.Declare("""
bool is_fiducial_y(const std::tuple<double,double,double> &vtx) {
    auto y = std::get<1>(vtx);
    auto vertical = (y < 23 && y > 15) || (y < 33 && y > 25) || (y < 44 && y > 36) || (y < 53 && y > 45);
    auto horizontal = (y < 20 && y > 12) || (y < 32 && y > 24) || (y < 44 && y > 36) || (y < 56 && y > 48);
    return horizontal && vertical;
}
""")

In [ ]:
# JIT a C++ function from Python
ROOT.gInterpreter.Declare("""
bool is_fiducial_z(const std::tuple<double,double,double> &vtx) {
    return std::get<2>(vtx) < -90;  // cm
}
""")

In [ ]:
# JIT a C++ function from Python
ROOT.gInterpreter.Declare("""
bool is_fiducial(const std::tuple<double,double,double> &vtx) {
    return is_fiducial_x(vtx) && is_fiducial_y(vtx) && is_fiducial_z(vtx);
}
""")

In [ ]:
# df_filtered = df.Filter("is_fiducial(Vertex)", "Vertex in Fiducial volume")
df = df.Define("fiducial_cut", "is_fiducial(Vertex)")

In [ ]:
df_filtered = df.Filter("std::size(SecondaryMuons) == 1", "Secondary muon").Filter(
    "is_fiducial(Vertex)", "Vertex in Fiducial volume"
)

In [ ]:
r = df_filtered.Report()

In [ ]:
r.Print()

In [ ]:
# columns = set(columns_orig)

In [ ]:
# for c in columns_orig:
#    if "Digi" in str(c):
#        columns.discard(c)

In [ ]:
df.Snapshot("cbmsim", "numu_golden.root", ["fiducial_cut", "CC_cut"])

In [ ]:
print(df.Sum("fiducial_cut").GetValue())

In [ ]:
print(df.Sum("CC_cut").GetValue())

In [ ]:
df = df.Define("all_cuts", "fiducial_cut && CC_cut")

In [ ]:
print(df.Sum("all_cuts").GetValue())